In [1]:
import pandas as pd
from config.config import SQLQuery

In [2]:
query_obj = SQLQuery('snowflake')
dataset_final = query_obj(r"""SELECT * FROM  PROD_DB.ADHOC.PILOT_DATASET_3 where signed_on_x is not null""")
dataset_final.to_csv('./dataset_pilot_final.csv', index=False)

In [3]:
dataset_final = pd.read_csv('./dataset_pilot_final.csv')

In [4]:
# placeholder= '%s'
# placeholders= ', '.join(placeholder for _ in dataset_final.business_id)
query= f"""
SELECT txn.id as transaction_id, txn.BUSINESS_ID, AMOUNT, DESCRIPTION, TYPE, MEDIUM, STATUS,
            TRANSACTION_DATE, RUNNING_BALANCE
            FROM  FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" txn
where TRANSACTION_DATE BETWEEN %s and %s
      and txn.status='active'
""" 

In [5]:
min_date = pd.to_datetime(dataset_final.signed_on_x).min() - pd.DateOffset(days=180)
max_date = pd.to_datetime(dataset_final.signed_on_x).max()

In [6]:
# query_obj = SQLQuery('snowflake')
# txn_df = query_obj(query, params=tuple([str(min_date), str(max_date)]))

In [7]:
# txn_df.to_csv('txn_details.csv', index=False)

In [8]:
# txn_df.shape

In [9]:
txn_df = pd.read_csv('txn_details.csv')

In [10]:
dataset_final = dataset_final[['business_id', 'email_sent_at', 'signed_on_x', 'fpd_plus_3', 'past_1_month']]

In [11]:
dataset_final['cutoff_date'] = pd.to_datetime(dataset_final.signed_on_x).dt.date

In [12]:
finaL_df = dataset_final.merge(txn_df, on='business_id', how='left', suffixes=['_business', '_txn'])

In [13]:
finaL_df['transaction_date'] = pd.to_datetime(finaL_df['transaction_date'])
finaL_df['transaction_date'] = finaL_df['transaction_date'].dt.date
finaL_df = finaL_df[finaL_df['transaction_date'] < finaL_df['cutoff_date']]
finaL_df = finaL_df[finaL_df['transaction_date'] >= (finaL_df['cutoff_date'] - pd.DateOffset(days=180))]

In [14]:
final_df_180 = finaL_df
final_df_90 = finaL_df[finaL_df['transaction_date'] >= (finaL_df['cutoff_date'] - pd.DateOffset(days=90))]

In [15]:
vals = ['AMAZON', 'AMZN', 'STRIPE', 'SQUARE INC', 'SHOPIFY', 'SHOPPAY', 'BILL',
               'INTUIT', 'WAVE', 'ETSY', 'UPWORK', 'WIX', 'POSHMARK', 'UBER', 'LYFT',
               'Doordash']
not_vals = ['LYFT', 'OfferUp', 'Gumroad', 'FB&Fundrai', 'Verify', 'CASH',
                   'PAYROLL', 'VRFY', 'CAPITAL', 'REFUND']
final_df_180['pd_deposits'] = ((final_df_180['type'] == 'credit') &
                                           (final_df_180['medium'] == 'External Deposit') &
                                           final_df_180['description'].str.contains('|'.join(vals), case=False,
                                                                                        na=False) &
                                           ~(final_df_180['description'].str.contains('|'.join(not_vals),
                                                                                          case=False,
                                                                                          na=True)))
final_df_90['pd_deposits'] = ((final_df_90['type'] == 'credit') &
                                             (final_df_90['medium'] == 'External Deposit') &
                                             final_df_90['description'].str.contains('|'.join(vals), case=False,
                                                                                            na=False) &
                                             ~(final_df_90['description'].str.contains('|'.join(not_vals),
                                                                                              case=False,
                                                                                              na=True)))
final_df_90['pd_deposits'] = final_df_90['pd_deposits'].astype(int)
final_df_180['pd_deposits'] = final_df_180['pd_deposits'].astype(int)
final_df_90['pd_deposit_amt'] = final_df_90['amount'] * final_df_90['pd_deposits']
final_df_180['pd_deposit_amt'] =final_df_180['amount'] * final_df_180['pd_deposits']

/var/folders/c3/mkqvfw9x5fs7djqx3_t8t6fr0000gn/T/ipykernel_77570/1287355307.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_90['pd_deposits'] = ((final_df_90['type'] == 'credit') &
/var/folders/c3/mkqvfw9x5fs7djqx3_t8t6fr0000gn/T/ipykernel_77570/1287355307.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_90['pd_deposits'] = final_df_90['pd_deposits'].astype(int)
/var/folders/c3/mkqvfw9x5fs7djqx3_t8t6fr0000gn/T/ipykernel_77570/1287355307.py:22: SettingWithCopyWarning: 
A value is t

In [16]:
features_df = dataset_final.copy()

In [17]:
features_df['date_of_refresh'] = pd.Timestamp.now()

In [18]:
pd_agg_180 = final_df_180.groupby('business_id').agg({'pd_deposits': 'sum', 'pd_deposit_amt': 'sum'}).reset_index()
pd_agg_90 = final_df_90.groupby('business_id').agg({'pd_deposits': 'sum', 'pd_deposit_amt': 'sum'}).reset_index()
pd_agg_180_90 = pd_agg_180.merge(pd_agg_90, on='business_id', how='left', suffixes=['_180', '_90'] ).fillna(0)

In [19]:
pd_agg_180_90['is_pd'] = (pd_agg_180_90['pd_deposits_180'] > 0).astype(int)

In [20]:
pd_agg_180_90['ratio_pd_freq_90_180'] = (pd_agg_180_90['pd_deposits_90'] / pd_agg_180_90['pd_deposits_180']).fillna(0)

In [21]:
pd_agg_180_90['ratio_pd_amt_90_180'] = (pd_agg_180_90['pd_deposit_amt_90'] / pd_agg_180_90['pd_deposit_amt_180']).fillna(0)

In [22]:
features_df = features_df.merge(pd_agg_180_90[['business_id', 'is_pd', 'ratio_pd_freq_90_180', 'ratio_pd_amt_90_180']], 
                                on='business_id', how='left')

In [23]:
ach_credits_180 = final_df_180[(final_df_180['type'] == 'credit') &
                               (final_df_180['medium'].str.contains('external deposit|iat deposit',
                               case=False))]
ach_credits_90 = final_df_90[(final_df_90['type'] == 'credit') &
                             (final_df_90['medium'].str.contains('external deposit|iat deposit',
                             case=False))]

In [24]:
ach_c_agg_180 = ach_credits_180.groupby('business_id').agg({'amount': 'sum', 'transaction_id': 'count'}).reset_index()
ach_c_agg_90 = ach_credits_90.groupby('business_id').agg({'amount': 'sum', 'transaction_id': 'count'}).reset_index()
ach_c_agg_180_90 = ach_c_agg_180.merge(ach_c_agg_90, on='business_id', how='left', suffixes=['_180', '_90'] ).fillna(0)

In [25]:
ach_c_agg_180_90['is_ach_credit'] = (ach_c_agg_180_90['transaction_id_180'] > 0).astype(int)
ach_c_agg_180_90['ratio_ach_credit_freq_90_180'] = (ach_c_agg_180_90['transaction_id_90'] / ach_c_agg_180_90['transaction_id_180']).fillna(0)
ach_c_agg_180_90['ratio_ach_credit_amt_90_180'] = (ach_c_agg_180_90['amount_90'] / ach_c_agg_180_90['amount_180']).fillna(0)

In [26]:
features_df = features_df.merge(ach_c_agg_180_90[['business_id', 'is_ach_credit', 'ratio_ach_credit_freq_90_180', 'ratio_ach_credit_amt_90_180']], 
                                on='business_id', how='left')

In [27]:
ach_debits_180 = final_df_180[(final_df_180['type'] == 'debit') &
                               (final_df_180['medium'].str.contains('external withdrawal|iat withdrawal',
                               case=False))]
ach_debits_90 = final_df_90[(final_df_90['type'] == 'debit') &
                             (final_df_90['medium'].str.contains('external withdrawal|iat withdrawal',
                             case=False))]

In [28]:
ach_d_agg_180 = ach_debits_180.groupby('business_id').agg({'amount': 'sum', 'transaction_id': 'count'}).reset_index()
ach_d_agg_90 = ach_debits_90.groupby('business_id').agg({'amount': 'sum', 'transaction_id': 'count'}).reset_index()
ach_d_agg_180_90 = ach_d_agg_180.merge(ach_d_agg_90, on='business_id', how='left', suffixes=['_180', '_90'] ).fillna(0)

In [29]:
ach_d_agg_180_90['is_ach_debit'] = (ach_d_agg_180_90['transaction_id_180'] > 0).astype(int)
ach_d_agg_180_90['ratio_ach_debit_freq_90_180'] = (ach_d_agg_180_90['transaction_id_90'] / ach_d_agg_180_90['transaction_id_180']).fillna(0)
ach_d_agg_180_90['ratio_ach_debit_amt_90_180'] = (ach_d_agg_180_90['amount_90'] / ach_d_agg_180_90['amount_180']).fillna(0)

In [30]:
features_df = features_df.merge(ach_d_agg_180_90[['business_id', 'is_ach_debit', 'ratio_ach_debit_freq_90_180', 'ratio_ach_debit_amt_90_180']], 
                                on='business_id', how='left')

In [31]:
mrdc_credits_180 = final_df_180[(final_df_180['type'] == 'credit') &
                               (final_df_180['description'] == 'Mobile Check Deposit')]
mrdc_credits_90 = final_df_90[(final_df_90['type'] == 'credit') &
                             (final_df_90['description'] == 'Mobile Check Deposit')]

In [32]:
mrdc_c_agg_180 = mrdc_credits_180.groupby('business_id').agg({'amount': 'sum', 'transaction_id': 'count'}).reset_index()
mrdc_c_agg_90 = mrdc_credits_90.groupby('business_id').agg({'amount': 'sum', 'transaction_id': 'count'}).reset_index()
mrdc_c_agg_180_90 = mrdc_c_agg_180.merge(mrdc_c_agg_90, on='business_id', how='left', suffixes=['_180', '_90'] ).fillna(0)

In [33]:
mrdc_c_agg_180_90['is_mrdc'] = (mrdc_c_agg_180_90['transaction_id_180'] > 0).astype(int)
mrdc_c_agg_180_90['ratio_mrdc_freq_90_180'] = (mrdc_c_agg_180_90['transaction_id_90'] / mrdc_c_agg_180_90['transaction_id_180']).fillna(0)
mrdc_c_agg_180_90['ratio_mrdc_amt_90_180'] = (mrdc_c_agg_180_90['amount_90'] / mrdc_c_agg_180_90['amount_180']).fillna(0)

In [34]:
features_df = features_df.merge(mrdc_c_agg_180_90[['business_id', 'is_mrdc', 'ratio_mrdc_freq_90_180', 'ratio_mrdc_amt_90_180']], 
                                on='business_id', how='left')

In [35]:
debits_180_agg = final_df_180[final_df_180['type'] == 'debit'].groupby(['business_id']).agg(
    {'amount': 'sum', 'transaction_id': 'count'}).reset_index().rename({
    'amount': 'total_amount', 'transaction_id': 'frequency'}, axis=1)
credits_180_agg = final_df_180[final_df_180['type'] == 'credit'].groupby(['business_id']).agg(
    {'amount': 'sum', 'transaction_id': 'count'}).reset_index().rename({
    'amount': 'total_amount', 'transaction_id': 'frequency'}, axis=1)
debit_credit_180 = debits_180_agg.merge(credits_180_agg, how='outer', on='business_id',
                                        suffixes=['_debit', '_credit']).fillna(0)
debit_credit_180['ratio_debit_credit_180'] = (-debit_credit_180['total_amount_debit'] / debit_credit_180['total_amount_credit']).fillna(0)

In [36]:
debit_credit_180['is_credit'] = (debit_credit_180['frequency_credit'] > 0).astype(int)
debit_credit_180['is_debit'] = (debit_credit_180['frequency_debit'] > 0).astype(int)
debit_credit_180['ratio_debit_credit_frequency'] = (debit_credit_180['frequency_debit']/ debit_credit_180['frequency_credit']).fillna(0)

In [37]:
features_df = features_df.merge(debit_credit_180[['business_id', 'ratio_debit_credit_180', 'is_credit',	'is_debit',	'ratio_debit_credit_frequency']], 
                                on='business_id', how='left')

In [38]:
card_debits_180 = final_df_180[final_df_180['medium'] == 'POS Withdrawal']
card_debits_90 = final_df_90[final_df_90['medium'] == 'POS Withdrawal']

In [39]:
card_debits_agg_180 = card_debits_180.groupby('business_id').agg({'amount': 'sum', 'transaction_id': 'count'}).reset_index()
card_debits_agg_90 = card_debits_90.groupby('business_id').agg({'amount': 'sum', 'transaction_id': 'count'}).reset_index()
card_debits_agg_180_90 = card_debits_agg_180.merge(card_debits_agg_90, on='business_id', how='left', suffixes=['_180', '_90'] ).fillna(0)

In [40]:
card_debits_agg_180_90['is_card_usage'] = (card_debits_agg_180_90['transaction_id_180'] > 0).astype(int)
card_debits_agg_180_90['ratio_card_withdrawal_freq_90_180'] = (card_debits_agg_180_90['transaction_id_90'] / card_debits_agg_180_90['transaction_id_180']).fillna(0)
card_debits_agg_180_90['ratio_card_withdrawal_amt_90_180'] = (card_debits_agg_180_90['amount_90'] / card_debits_agg_180_90['amount_180']).fillna(0)

In [41]:
card_debits_agg_180_90 = card_debits_agg_180_90.fillna(0)

In [42]:
features_df = features_df.merge(card_debits_agg_180_90[['business_id', 'is_card_usage', 'ratio_card_withdrawal_freq_90_180', 'ratio_card_withdrawal_amt_90_180']], 
                                on='business_id', how='left')

In [43]:
query_obj = SQLQuery('snowflake')
all_credits_txn = query_obj(r"""
WITH txn as (
  		SELECT id AS transaction_id, BUSINESS_ID , amount, TYPE, TRANSACTION_DATE,  
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
WHERE signed_on_X IS NOT NULL AND  status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),

oneMonth_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_credited_1M,
AVG(AMOUNT) as Avg_amount_credited_1M,
Median(AMOUNT) as Median_amount_credited_1M,
STDDEV(AMOUNT) as STDDEV_amount_credited_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_credit_txns_1M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as credit_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND TYPE = 'credit' 
GROUP BY BUSINESS_ID),


threeMonths_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_credited_3M,
AVG(AMOUNT) as Avg_amount_credited_3M,
Median(AMOUNT) as Median_amount_credited_3M,
STDDEV(AMOUNT) as STDDEV_amount_credited_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_credit_txns_3M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as credit_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND TYPE = 'credit' GROUP BY BUSINESS_ID),

sixMonths_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_credited_6M,
AVG(AMOUNT) as Avg_amount_credited_6M,
Median(AMOUNT) as Median_amount_credited_6M,
STDDEV(AMOUNT) as STDDEV_amount_credited_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_credit_txns_6M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as credit_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND TYPE = 'credit' GROUP BY BUSINESS_ID),

aggregate_1_3_6_credit as (
SELECT * 
FROM
sixMonths_credit AS a
FULL OUTER JOIN 
threeMonths_credit AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_credit AS c
USING (BUSINESS_ID))

SELECT * FROM aggregate_1_3_6_credit
""")

In [44]:
features_df = features_df.merge(all_credits_txn, on='business_id', how='left')

In [45]:
query_obj = SQLQuery('snowflake')
all_debits_txn = query_obj(r"""
WITH txn as (
  		SELECT id AS transaction_id, BUSINESS_ID , amount, TYPE, TRANSACTION_DATE,  
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
WHERE signed_on_X IS NOT NULL AND  status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),
oneMonth_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_debited_1M,
AVG(-AMOUNT) as Avg_amount_debited_1M,
Median(-AMOUNT) as Median_amount_debited_1M,
STDDEV(-AMOUNT) as STDDEV_amount_debited_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_debit_txns_1M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as debit_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND TYPE = 'debit' 
GROUP BY BUSINESS_ID),


threeMonths_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_debited_3M,
AVG(-AMOUNT) as Avg_amount_debited_3M,
Median(-AMOUNT) as Median_amount_debited_3M,
STDDEV(-AMOUNT) as STDDEV_amount_debited_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_debit_txns_3M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as debit_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND TYPE = 'debit' GROUP BY BUSINESS_ID),

sixMonths_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_debited_6M,
AVG(-AMOUNT) as Avg_amount_debited_6M,
Median(-AMOUNT) as Median_amount_debited_6M,
STDDEV(-AMOUNT) as STDDEV_amount_debited_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_debit_txns_6M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as debit_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND TYPE = 'debit' GROUP BY BUSINESS_ID),

aggregate_1_3_6_debit as (
SELECT * 
FROM
sixMonths_debit AS a
FULL OUTER JOIN 
threeMonths_debit AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_debit AS c
USING (BUSINESS_ID))

SELECT * FROM aggregate_1_3_6_debit
""")

In [46]:
features_df = features_df.merge(all_debits_txn, on='business_id', how='left')

In [47]:
query_obj = SQLQuery('snowflake')
all_pd_txn = query_obj(r"""
WITH txn as (
  		SELECT id AS transaction_id, BUSINESS_ID , amount, TYPE, TRANSACTION_DATE,  
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
WHERE signed_on_X IS NOT NULL AND  status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),

oneMonth_pd as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_pd_1M,
AVG(AMOUNT) as Avg_amount_pd_1M,
Median(AMOUNT) as Median_amount_pd_1M,
STDDEV(AMOUNT) as STDDEV_amount_pd_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_1M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as pd_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND PD_DEPOSIT_TXN = 1 
GROUP BY BUSINESS_ID),

oneMonth_pd_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_100_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND PD_DEPOSIT_TXN = 1 AND amount > 100
GROUP BY BUSINESS_ID),

oneMonth_pd_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_1000_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND PD_DEPOSIT_TXN = 1 AND amount > 1000
GROUP BY BUSINESS_ID),

threeMonths_pd as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_pd_3M,
AVG(AMOUNT) as Avg_amount_pd_3M,
Median(AMOUNT) as Median_amount_pd_3M,
STDDEV(AMOUNT) as STDDEV_amount_pd_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_3M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as pd_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  PD_DEPOSIT_TXN = 1  
GROUP BY BUSINESS_ID),

threeMonths_pd_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_100_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND PD_DEPOSIT_TXN = 1 AND amount > 100
GROUP BY BUSINESS_ID),

threeMonths_pd_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_1000_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND PD_DEPOSIT_TXN = 1 AND amount > 1000
GROUP BY BUSINESS_ID),

sixMonths_pd as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_pd_6M,
AVG(AMOUNT) as Avg_amount_pd_6M,
Median(AMOUNT) as Median_amount_pd_6M,
STDDEV(AMOUNT) as STDDEV_amount_pd_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_6M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as pd_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  PD_DEPOSIT_TXN = 1 
GROUP BY BUSINESS_ID),

sixMonths_pd_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_100_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND PD_DEPOSIT_TXN = 1 AND amount > 100
GROUP BY BUSINESS_ID),

sixMonths_pd_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_pd_txns_1000_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND PD_DEPOSIT_TXN = 1 AND amount > 1000
GROUP BY BUSINESS_ID),

aggregate_1_3_6_pd as (
SELECT * 
FROM
sixMonths_pd AS a
FULL OUTER JOIN 
threeMonths_pd AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_pd AS c
USING (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_pd_100 AS d
USING (business_id)
FULL OUTER JOIN 
oneMonth_pd_1000 AS e
using(business_id)
FULL OUTER JOIN 
threeMonths_pd_100 AS f
USING (business_id)
FULL OUTER JOIN 
threeMonths_pd_1000 AS g
using(business_id)
FULL OUTER JOIN 
sixMonths_pd_100 AS h
USING (business_id)
FULL OUTER JOIN 
sixMonths_pd_1000 AS i
using(business_id))

SELECT * FROM aggregate_1_3_6_pd 
""")

In [48]:
features_df = features_df.merge(all_pd_txn, on='business_id', how='left')

In [49]:
query_obj = SQLQuery('snowflake')
all_ach_c_txn = query_obj(r"""
WITH txn as (
  		SELECT id AS transaction_id, BUSINESS_ID , amount, TYPE, MEDIUM, TRANSACTION_DATE,  
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
WHERE signed_on_X IS NOT NULL AND  status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),

oneMonth_ach_c as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_ach_c_1M,
AVG(AMOUNT) as Avg_amount_ach_c_1M,
Median(AMOUNT) as Median_amount_ach_c_1M,
STDDEV(AMOUNT) as STDDEV_amount_ach_c_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_1M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as ach_c_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%')
GROUP BY BUSINESS_ID),

oneMonth_ach_c_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_100_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') AND amount > 100
GROUP BY BUSINESS_ID),

oneMonth_ach_c_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_1000_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') AND amount > 1000
GROUP BY BUSINESS_ID),

threeMonths_ach_c as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_ach_c_3M,
AVG(AMOUNT) as Avg_amount_ach_c_3M,
Median(AMOUNT) as Median_amount_ach_c_3M,
STDDEV(AMOUNT) as STDDEV_amount_ach_c_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_3M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as ach_c_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%')
GROUP BY BUSINESS_ID),

threeMonths_ach_c_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_100_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') AND amount > 100
GROUP BY BUSINESS_ID),

threeMonths_ach_c_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_1000_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') AND amount > 1000
GROUP BY BUSINESS_ID),

sixMonths_ach_c as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_ach_c_6M,
AVG(AMOUNT) as Avg_amount_ach_c_6M,
Median(AMOUNT) as Median_amount_ach_c_6M,
STDDEV(AMOUNT) as STDDEV_amount_ach_c_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_6M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as ach_c_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%')
GROUP BY BUSINESS_ID),

sixMonths_ach_c_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_100_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') AND amount > 100
GROUP BY BUSINESS_ID),

sixMonths_ach_c_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_c_txns_1000_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') AND amount > 1000
GROUP BY BUSINESS_ID),

aggregate_1_3_6_ach_c as (
SELECT * 
FROM
sixMonths_ach_c AS a
FULL OUTER JOIN 
threeMonths_ach_c AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_ach_c AS c
USING (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_ach_c_100 AS d
USING (business_id)
FULL OUTER JOIN 
oneMonth_ach_c_1000 AS e
using(business_id)
FULL OUTER JOIN 
threeMonths_ach_c_100 AS f
USING (business_id)
FULL OUTER JOIN 
threeMonths_ach_c_1000 AS g
using(business_id)
FULL OUTER JOIN 
sixMonths_ach_c_100 AS h
USING (business_id)
FULL OUTER JOIN 
sixMonths_ach_c_1000 AS i
using(business_id))

SELECT * FROM aggregate_1_3_6_ach_c 
""")

In [50]:
features_df = features_df.merge(all_ach_c_txn, on='business_id', how='left')

In [51]:
query_obj = SQLQuery('snowflake')
all_ach_d_txn = query_obj(r"""
WITH txn as (
  		SELECT id AS transaction_id, BUSINESS_ID , amount, TYPE, MEDIUM, TRANSACTION_DATE,  
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
WHERE signed_on_X IS NOT NULL AND  status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),
oneMonth_ach_d as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_ach_d_1M,
AVG(-AMOUNT) as Avg_amount_ach_d_1M,
Median(-AMOUNT) as Median_amount_ach_d_1M,
STDDEV(-AMOUNT) as STDDEV_amount_ach_d_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_1M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as ach_d_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%')
GROUP BY BUSINESS_ID),

oneMonth_ach_d_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_100_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%') AND -amount > 100
GROUP BY BUSINESS_ID),

oneMonth_ach_d_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_1000_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%') AND -amount > 1000
GROUP BY BUSINESS_ID),

threeMonths_ach_d as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_ach_d_3M,
AVG(-AMOUNT) as Avg_amount_ach_d_3M,
Median(-AMOUNT) as Median_amount_ach_d_3M,
STDDEV(-AMOUNT) as STDDEV_amount_ach_d_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_3M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as ach_d_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%')
GROUP BY BUSINESS_ID),

threeMonths_ach_d_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_100_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%') AND -amount > 100
GROUP BY BUSINESS_ID),

threeMonths_ach_d_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_1000_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%') AND -amount > 1000
GROUP BY BUSINESS_ID),

sixMonths_ach_d as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_ach_d_6M,
AVG(-AMOUNT) as Avg_amount_ach_d_6M,
Median(-AMOUNT) as Median_amount_ach_d_6M,
STDDEV(-AMOUNT) as STDDEV_amount_ach_d_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_6M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as ach_d_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%')
GROUP BY BUSINESS_ID),

sixMonths_ach_d_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_100_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%') AND -amount > 100
GROUP BY BUSINESS_ID),

sixMonths_ach_d_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_ach_d_txns_1000_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND type = 'debit'  AND medium ILIKE ANY('%external withdrawal%', '%iat withdrawal%') AND -amount > 1000
GROUP BY BUSINESS_ID),

aggregate_1_3_6_ach_d as (
SELECT * 
FROM
sixMonths_ach_d AS a
FULL OUTER JOIN 
threeMonths_ach_d AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_ach_d AS c
USING (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_ach_d_100 AS d
USING (business_id)
FULL OUTER JOIN 
oneMonth_ach_d_1000 AS e
using(business_id)
FULL OUTER JOIN 
threeMonths_ach_d_100 AS f
USING (business_id)
FULL OUTER JOIN 
threeMonths_ach_d_1000 AS g
using(business_id)
FULL OUTER JOIN 
sixMonths_ach_d_100 AS h
USING (business_id)
FULL OUTER JOIN 
sixMonths_ach_d_1000 AS i
using(business_id))

SELECT * FROM aggregate_1_3_6_ach_d 
""")

In [52]:
features_df = features_df.merge(all_ach_d_txn, on='business_id', how='left')

In [53]:
query_obj = SQLQuery('snowflake')
all_mrdc_txn = query_obj(r"""
WITH txn as (
  		SELECT id AS transaction_id, BUSINESS_ID , amount, TYPE, MEDIUM, description, TRANSACTION_DATE,  
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
WHERE signed_on_X IS NOT NULL AND  status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),

oneMonth_mrdc as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_mrdc_1M,
AVG(AMOUNT) as Avg_amount_mrdc_1M,
Median(AMOUNT) as Median_amount_mrdc_1M,
STDDEV(AMOUNT) as STDDEV_amount_mrdc_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_1M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as mrdc_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'credit'  AND description = 'Mobile Check Deposit'
GROUP BY BUSINESS_ID),

oneMonth_mrdc_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_100_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'credit'  AND description = 'Mobile Check Deposit' AND amount > 100
GROUP BY BUSINESS_ID),

oneMonth_mrdc_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_1000_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND type = 'credit'  AND description = 'Mobile Check Deposit' AND amount > 1000
GROUP BY BUSINESS_ID),

threeMonths_mrdc as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_mrdc_3M,
AVG(AMOUNT) as Avg_amount_mrdc_3M,
Median(AMOUNT) as Median_amount_mrdc_3M,
STDDEV(AMOUNT) as STDDEV_amount_mrdc_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_3M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as mrdc_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  type = 'credit'  AND description = 'Mobile Check Deposit'
GROUP BY BUSINESS_ID),

threeMonths_mrdc_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_100_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND type = 'credit'  AND description = 'Mobile Check Deposit' AND amount > 100
GROUP BY BUSINESS_ID),

threeMonths_mrdc_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_1000_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND type = 'credit'  AND description = 'Mobile Check Deposit' AND amount > 1000
GROUP BY BUSINESS_ID),

sixMonths_mrdc as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_mrdc_6M,
AVG(AMOUNT) as Avg_amount_mrdc_6M,
Median(AMOUNT) as Median_amount_mrdc_6M,
STDDEV(AMOUNT) as STDDEV_amount_mrdc_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_6M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as mrdc_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  type = 'credit'  AND description = 'Mobile Check Deposit'
GROUP BY BUSINESS_ID),

sixMonths_mrdc_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_100_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND type = 'credit'  AND description = 'Mobile Check Deposit' AND amount > 100
GROUP BY BUSINESS_ID),

sixMonths_mrdc_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_mrdc_txns_1000_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND type = 'credit'  AND description = 'Mobile Check Deposit' AND amount > 1000
GROUP BY BUSINESS_ID),

aggregate_1_3_6_mrdc as (
SELECT * 
FROM
sixMonths_mrdc AS a
FULL OUTER JOIN 
threeMonths_mrdc AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_mrdc AS c
USING (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_mrdc_100 AS d
USING (business_id)
FULL OUTER JOIN 
oneMonth_mrdc_1000 AS e
using(business_id)
FULL OUTER JOIN 
threeMonths_mrdc_100 AS f
USING (business_id)
FULL OUTER JOIN 
threeMonths_mrdc_1000 AS g
using(business_id)
FULL OUTER JOIN 
sixMonths_mrdc_100 AS h
USING (business_id)
FULL OUTER JOIN 
sixMonths_mrdc_1000 AS i
using(business_id))

SELECT * FROM aggregate_1_3_6_mrdc
""")

In [54]:
features_df = features_df.merge(all_mrdc_txn, on='business_id', how='left')

In [55]:
query_obj = SQLQuery('snowflake')
all_card_txn = query_obj(r"""
WITH txn as (
  		SELECT id AS transaction_id, BUSINESS_ID , amount, TYPE, MEDIUM, description, TRANSACTION_DATE,  
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
WHERE signed_on_X IS NOT NULL AND  status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),

oneMonth_card as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_card_1M,
AVG(-AMOUNT) as Avg_amount_card_1M,
Median(-AMOUNT) as Median_amount_card_1M,
STDDEV(-AMOUNT) as STDDEV_amount_card_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_1M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as card_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND medium= 'POS Withdrawal'
GROUP BY BUSINESS_ID),

oneMonth_card_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_100_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND medium= 'POS Withdrawal' AND -amount > 100
GROUP BY BUSINESS_ID),

oneMonth_card_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_1000_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND medium= 'POS Withdrawal' AND -amount > 1000
GROUP BY BUSINESS_ID),

threeMonths_card as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_card_3M,
AVG(-AMOUNT) as Avg_amount_card_3M,
Median(-AMOUNT) as Median_amount_card_3M,
STDDEV(-AMOUNT) as STDDEV_amount_card_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_3M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as card_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  medium= 'POS Withdrawal'
GROUP BY BUSINESS_ID),

threeMonths_card_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_100_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND medium= 'POS Withdrawal' AND -amount > 100
GROUP BY BUSINESS_ID),

threeMonths_card_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_1000_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND medium= 'POS Withdrawal' AND -amount > 1000
GROUP BY BUSINESS_ID),

sixMonths_card as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_card_6M,
AVG(-AMOUNT) as Avg_amount_card_6M,
Median(-AMOUNT) as Median_amount_card_6M,
STDDEV(-AMOUNT) as STDDEV_amount_card_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_6M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as card_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  medium= 'POS Withdrawal'
GROUP BY BUSINESS_ID),

sixMonths_card_100 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_100_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND medium= 'POS Withdrawal' AND -amount > 100
GROUP BY BUSINESS_ID),

sixMonths_card_1000 as (SELECT 
BUSINESS_ID as BUSINESS_ID,
COUNT(DISTINCT TRANSACTION_ID) as distinct_card_txns_1000_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND medium= 'POS Withdrawal' AND -amount > 1000
GROUP BY BUSINESS_ID),

aggregate_1_3_6_card as (
SELECT * 
FROM
sixMonths_card AS a
FULL OUTER JOIN 
threeMonths_card AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_card AS c
USING (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_card_100 AS d
USING (business_id)
FULL OUTER JOIN 
oneMonth_card_1000 AS e
using(business_id)
FULL OUTER JOIN 
threeMonths_card_100 AS f
USING (business_id)
FULL OUTER JOIN 
threeMonths_card_1000 AS g
using(business_id)
FULL OUTER JOIN 
sixMonths_card_100 AS h
USING (business_id)
FULL OUTER JOIN 
sixMonths_card_1000 AS i
using(business_id))

SELECT * FROM aggregate_1_3_6_card 
""")

In [56]:
features_df = features_df.merge(all_card_txn, on='business_id', how='left')

In [57]:
features_df.iloc[:, 8:] = features_df.iloc[:, 8:].fillna(0)

In [58]:
features_df['ratio_pd_amt_total_credit_1m'] = (features_df['amount_pd_1m'] / features_df['amount_credited_1m']).fillna(0)
features_df['ratio_pd_amt_total_credit_3m'] = (features_df['amount_pd_3m'] / features_df['amount_credited_3m']).fillna(0)
features_df['ratio_pd_amt_total_credit_6m'] = (features_df['amount_pd_6m'] / features_df['amount_credited_6m']).fillna(0)

In [59]:
features_df['ratio_pd_freq_total_credit_1m'] = (features_df['distinct_pd_txns_1m']/features_df['distinct_credit_txns_1m']).fillna(0)
features_df['ratio_pd_freq_total_credit_3m'] = (features_df['distinct_pd_txns_3m']/features_df['distinct_credit_txns_3m']).fillna(0)
features_df['ratio_pd_freq_total_credit_6m'] = (features_df['distinct_pd_txns_6m']/features_df['distinct_credit_txns_6m']).fillna(0)

In [60]:
features_df['ratio_card_amt_total_debit_1m'] = (features_df['amount_card_1m']/features_df['amount_debited_1m']).fillna(0)
features_df['ratio_card_amt_total_debit_3m'] = (features_df['amount_card_3m']/features_df['amount_debited_3m']).fillna(0)
features_df['ratio_card_amt_total_debit_6m'] = (features_df['amount_card_6m']/features_df['amount_debited_6m']).fillna(0)

In [61]:
features_df['ratio_mrdc_amt_total_credit_1m'] = (features_df['amount_mrdc_1m']/features_df['amount_credited_1m']).fillna(0)
features_df['ratio_mrdc_amt_total_credit_3m'] = (features_df['amount_mrdc_3m']/features_df['amount_credited_3m']).fillna(0)
features_df['ratio_mrdc_amt_total_credit_6m'] = (features_df['amount_mrdc_6m']/features_df['amount_credited_6m']).fillna(0)

In [62]:
features_df['ratio_ach_amt_debit_credit_1m'] = (features_df['amount_ach_d_1m']/features_df['amount_ach_c_1m']).fillna(0)
features_df['ratio_ach_amt_debit_credit_3m'] = (features_df['amount_ach_d_3m']/features_df['amount_ach_c_3m']).fillna(0)
features_df['ratio_ach_amt_debit_credit_6m'] = (features_df['amount_ach_d_6m']/features_df['amount_ach_c_6m']).fillna(0)

In [63]:
features_df['ratio_ach_freq_debit_credit_1m'] = (features_df['distinct_ach_d_txns_1m']/features_df['distinct_ach_c_txns_1m']).fillna(0)
features_df['ratio_ach_freq_debit_credit_3m'] = (features_df['distinct_ach_d_txns_3m']/features_df['distinct_ach_c_txns_3m']).fillna(0)
features_df['ratio_ach_freq_debit_credit_6m'] = (features_df['distinct_ach_d_txns_6m']/features_df['distinct_ach_c_txns_6m']).fillna(0)

In [64]:
features_df['ratio_total_freq_debit_credit_1m'] = (features_df['distinct_debit_txns_1m']/features_df['distinct_credit_txns_1m']).fillna(0)
features_df['ratio_total_freq_debit_credit_3m'] = (features_df['distinct_debit_txns_3m']/features_df['distinct_credit_txns_3m']).fillna(0)
features_df['ratio_total_freq_debit_credit_6m'] = (features_df['distinct_debit_txns_6m']/features_df['distinct_credit_txns_6m']).fillna(0)

In [65]:
features_df['ratio_debit_credit_1m'] = (features_df['amount_debited_1m']/features_df['amount_credited_1m']).fillna(0)
features_df['ratio_debit_credit_3m'] = (features_df['amount_debited_3m']/features_df['amount_credited_3m']).fillna(0)
features_df['ratio_debit_credit_6m'] = (features_df['amount_debited_3m']/features_df['amount_credited_6m']).fillna(0)

In [66]:
query_obj = SQLQuery('snowflake')
od_zero_balance = query_obj(r"""
with cal_dates_1 AS (
			SELECT DATEADD(DAY, SEQ4(), '2021-06-01') AS CAL_DATE
			 FROM TABLE(GENERATOR(ROWCOUNT=>640)) 
			 ORDER BY CAL_DATE DESC),
biz_dates_1 AS (
			SELECT a.BUSINESS_ID, MIN(a.transaction_date) AS FIRST_TXN_DATE
			from FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS"  a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
			GROUP BY a.BUSINESS_ID),
PILOT_BIZ_DATES_TEMP AS (
		SELECT *
		FROM  cal_dates_1 A LEFT JOIN
		biz_dates_1 B ON
		date(A.CAL_DATE) >= date(B.FIRST_TXN_DATE)
		ORDER BY CAL_DATE ASC)
,PILOT_BIZ_TXN_TEMP AS (
		SELECT   Business_id ,TRANSACTION_DATE,running_balance
		FROM
		        (SELECT *, RANK () OVER (PARTITION BY BUSINESS_ID,TRANSACTION_DATE ORDER BY TRANSACTION_DATE DESC) RANKS
		         FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" 
                 where 1=1
                 and status='active'
		         )
		WHERE RANKS=1
		order by Business_id, TRANSACTION_DATE)
,PILOT_DAILY_BALANCES_1  AS (
		SELECT BUSINESS_ID,FIRST_TXN_DATE, CAL_DATE,TRANSACTION_DATE,RUNNING_BALANCE,
		first_value(RUNNING_BALANCE) OVER (PARTITION BY BUSINESS_ID,GROUPER ORDER BY cal_date asc, TRANSACTION_DATE desc, RUNNING_BALANCE asc nulls last) as RUNNING_BALANCE_2
		FROM (
		SELECT A.BUSINESS_ID,A.CAL_DATE,A.FIRST_TXN_DATE,
		        B.TRANSACTION_DATE,B.RUNNING_BALANCE,
		         COUNT(running_balance) OVER (PARTITION BY a.BUSINESS_ID ORDER BY cal_date asc) as grouper
		        FROM PILOT_BIZ_DATES_TEMP A LEFT JOIN
		        PILOT_BIZ_TXN_TEMP B ON
		        A.BUSINESS_ID=B.BUSINESS_ID AND
		        date(A.CAL_DATE)= date(B.TRANSACTION_DATE)
		        ORDER BY A.CAL_DATE ASC)
		ORDER BY BUSINESS_ID,CAL_DATE)
,final_daily_running_balance AS (
		 SELECT distinct a.business_id, datediff(day, a.cal_date, signed_on_x) as daySinceLoanTaken, cal_date, running_balance_2 AS running_balance_daily
		           FROM  PILOT_DAILY_BALANCES_1  a   INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
		          WHERE daySinceLoanTaken > 0 AND daySinceLoanTaken <= 180
		ORDER BY a.business_id, daySinceLoanTaken)
,oneMonth_rb AS (	
				SELECT business_id, 
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_1m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_1m,
				               AVG(running_balance_daily) as Avg_running_balance_1M,
  							   Median(running_balance_daily) as Median_running_balance_1M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_1M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=30
				GROUP BY business_id),
threeMonths_rb AS (	
				SELECT business_id, 
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_3m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_3m,
				               AVG(running_balance_daily) as Avg_running_balance_3M,
  							   Median(running_balance_daily) as Median_running_balance_3M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_3M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=90
				GROUP BY business_id),
 sixMonths_rb AS (	
				SELECT business_id, 
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_6m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_6m,
				               AVG(running_balance_daily) as Avg_running_balance_6M,
  							   Median(running_balance_daily) as Median_running_balance_6M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_6M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=180
				GROUP BY business_id),
aggregate_1_3_6_rb as (
				SELECT * 
				FROM
				sixMonths_rb AS a
				FULL OUTER JOIN 
				threeMonths_rb AS b
				using (BUSINESS_ID)
				FULL OUTER JOIN 
				oneMonth_rb AS c
				USING (BUSINESS_ID))
SELECT * FROM aggregate_1_3_6_rb 
""")

In [67]:
od_zero_balance = od_zero_balance.fillna(0)

In [68]:
features_df = features_df.merge(od_zero_balance, on='business_id', how='left')

In [69]:
query_obj = SQLQuery('snowflake')
is_each_month = query_obj(r"""
WITH
   txn as (
  		SELECT id AS transaction_id, BUSINESS_ID , amount, TYPE, MEDIUM, description, TRANSACTION_DATE,  
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken,
             CASE WHEN daySinceLoanTaken > 0 and daySinceLoanTaken <=30 THEN 'Month 1' 
                  WHEN daySinceLoanTaken > 30 and daySinceLoanTaken <=60 THEN 'Month 2' 
                  WHEN daySinceLoanTaken > 60 and daySinceLoanTaken <=90 THEN 'Month 3' 
                  WHEN daySinceLoanTaken > 90 and daySinceLoanTaken <=120 THEN 'Month 4' 
                  WHEN daySinceLoanTaken > 120 and daySinceLoanTaken <=150 THEN 'Month 5' 
                  WHEN daySinceLoanTaken > 150 and daySinceLoanTaken <=180 THEN 'Month 6' 
             END as transaction_month
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 b using(business_id)
WHERE signed_on_X IS NOT NULL AND  status = 'active'  AND  daySinceLoanTaken <= 180 AND daySinceLoanTaken > 0
)
,is_transaction_in_month AS(
 SELECT business_id, transaction_month,
 				CASE WHEN SUM(card_txn) > 0 THEN 1 ELSE 0 END AS is_card_usage, 
 				CASE WHEN SUM(pd_txn) > 0 THEN 1 ELSE 0 END AS is_pd, 
 				CASE WHEN SUM(ach_credit_txn) > 0 THEN 1 ELSE 0 END AS is_ach_credit, 
 				CASE WHEN SUM(ach_debit_txn) >0 THEN 1 ELSE 0 END AS is_ach_debit, 
 				CASE WHEN SUM(mrdc_txn) > 0 THEN 1 ELSE 0 END AS is_mrdc
 FROM (
		SELECT BUSINESS_ID, transaction_month,
		                CASE WHEN sum(CASE WHEN medium= 'POS Withdrawal' THEN 1 ELSE 0 END) >= 1 THEN 1 ELSE 0 end AS card_txn,
		                CASE WHEN sum(PD_DEPOSIT_TXN) >= 1 THEN 1 ELSE 0 END AS pd_txn,
		                CASE WHEN sum(CASE WHEN type = 'credit'  AND medium ILIKE ANY ('%%external deposit%%', '%%iat deposit%%') THEN 1 ELSE 0 END) >=1 THEN 1 ELSE 0 END AS ach_credit_txn,
		                CASE WHEN sum(CASE WHEN type = 'debit'  AND medium ILIKE ANY ('%%external withdrawal%%', '%%iat withdrawal%%') THEN 1 ELSE 0 END) >=1 THEN 1 ELSE 0 END AS ach_debit_txn,
		                CASE WHEN sum(CASE WHEN type = 'credit'  AND description = 'Mobile Check Deposit' THEN 1 ELSE 0 END) >=1 THEN 1 ELSE 0 END AS mrdc_txn
		 FROM txn 
		 GROUP BY BUSINESS_ID, transaction_month
		 )
GROUP BY BUSINESS_ID , transaction_month
)
SELECT business_id,
 				CASE WHEN SUM(is_card_usage) = 6 THEN 1 ELSE 0 END AS is_card_usage_each_month, 
 				CASE WHEN SUM(is_pd) = 6 THEN 1 ELSE 0 END AS is_pd_each_month, 
 				CASE WHEN SUM(is_ach_credit) = 6 THEN 1 ELSE 0 END AS is_ach_credit_each_month, 
 				CASE WHEN SUM(is_ach_debit) =6 THEN 1 ELSE 0 END AS is_ach_debit_each_month, 
 				CASE WHEN SUM(is_mrdc) = 6 THEN 1 ELSE 0 END AS is_mrdc_each_month
 FROM is_transaction_in_month
 GROUP BY BUSINESS_ID
 order by business_id
""")

In [70]:
features_df = features_df.merge(is_each_month, on='business_id', how='left')

In [71]:
query_obj = SQLQuery('snowflake')
loan_credits = query_obj(r""" 
WITH txn as (
  		SELECT a.id AS transaction_id, a.BUSINESS_ID , a.amount, a.TYPE, a.MEDIUM, a.description, a.TRANSACTION_DATE,  
              CASE WHEN a.TYPE = 'credit' and a.DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    a.description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN a.TYPE ='debit' AND a.MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN a.type = 'credit'  AND a.medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN a.medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN a.TYPE = 'credit' AND a.description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, a.TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken, c.loans, c.insurance, c.tax
          FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a  
          JOIN PROD_DB.ADHOC.LENDING_NOVO_TXN_CATEGORIES_SAMPLE c ON c.transaction_id = a.id 
           INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 d on a.business_id = d.business_id
WHERE signed_on_X IS NOT NULL AND  a.status = 'active'  AND  daySinceLoanTaken <= 180 AND daySinceLoanTaken > 0
),

oneMonth_loan_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_loan_credit_1M,
AVG(AMOUNT) as Avg_amount_loan_credit_1M,
Median(AMOUNT) as Median_amount_loan_credit_1M,
STDDEV(AMOUNT) as STDDEV_amount_loan_credit_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_loan_credit_txns_1M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as loan_credit_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND loans = 1 AND TYPE='credit'
GROUP BY BUSINESS_ID),

threeMonths_loan_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_loan_credit_3M,
AVG(AMOUNT) as Avg_amount_loan_credit_3M,
Median(AMOUNT) as Median_amount_loan_credit_3M,
STDDEV(AMOUNT) as STDDEV_amount_loan_credit_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_loan_credit_txns_3M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as loan_credit_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  loans = 1 AND TYPE='credit'  
GROUP BY BUSINESS_ID),


sixMonths_loan_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_loan_credit_6M,
AVG(AMOUNT) as Avg_amount_loan_credit_6M,
Median(AMOUNT) as Median_amount_loan_credit_6M,
STDDEV(AMOUNT) as STDDEV_amount_loan_credit_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_loan_credit_txns_6M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as loan_credit_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  loans = 1 AND TYPE='credit'
GROUP BY BUSINESS_ID),

aggregate_1_3_6_loan_credit as (
SELECT * 
FROM
sixMonths_loan_credit AS a
FULL OUTER JOIN 
threeMonths_loan_credit AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_loan_credit AS c
USING (BUSINESS_ID))

SELECT * FROM aggregate_1_3_6_loan_credit 
""")

In [72]:
features_df = features_df.merge(loan_credits, on='business_id', how='left')

In [73]:
features_df[loan_credits.columns[1:]] = features_df[loan_credits.columns[1:]].fillna(0)

In [74]:
features_df['ratio_loan_credit_total_credit_1m'] = (features_df['amount_loan_credit_1m'] / features_df['amount_credited_1m']).fillna(0)
features_df['ratio_loan_credit_total_credit_3m'] = (features_df['amount_loan_credit_3m'] / features_df['amount_credited_3m']).fillna(0)
features_df['ratio_loan_credit_total_credit_6m'] = (features_df['amount_loan_credit_6m'] / features_df['amount_credited_6m']).fillna(0)

In [75]:
query_obj = SQLQuery('snowflake')
loan_debits = query_obj(r"""
WITH txn as (
  		SELECT a.id AS transaction_id, a.BUSINESS_ID , a.amount, a.TYPE, a.MEDIUM, a.description, a.TRANSACTION_DATE,  
              CASE WHEN a.TYPE = 'credit' and a.DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    a.description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN a.TYPE ='debit' AND a.MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN a.type = 'credit'  AND a.medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN a.medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN a.TYPE = 'credit' AND a.description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, a.TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken, c.loans, c.insurance, c.tax
          FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a  
          JOIN PROD_DB.ADHOC.LENDING_NOVO_TXN_CATEGORIES_SAMPLE c ON c.transaction_id = a.id 
           INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 d on a.business_id = d.business_id
WHERE signed_on_X IS NOT NULL AND  a.status = 'active'  AND  daySinceLoanTaken <= 180 AND daySinceLoanTaken > 0),
oneMonth_loan_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_loan_debit_1M,
AVG(-AMOUNT) as Avg_amount_loan_debit_1M,
Median(-AMOUNT) as Median_amount_loan_debit_1M,
STDDEV(-AMOUNT) as STDDEV_amount_loan_debit_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_loan_debit_txns_1M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as loan_debit_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND loans = 1 AND TYPE='debit'
GROUP BY BUSINESS_ID),

threeMonths_loan_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_loan_debit_3M,
AVG(-AMOUNT) as Avg_amount_loan_debit_3M,
Median(-AMOUNT) as Median_amount_loan_debit_3M,
STDDEV(-AMOUNT) as STDDEV_amount_loan_debit_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_loan_debit_txns_3M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as loan_debit_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  loans = 1 AND TYPE='debit'  
GROUP BY BUSINESS_ID),


sixMonths_loan_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_loan_debit_6M,
AVG(-AMOUNT) as Avg_amount_loan_debit_6M,
Median(-AMOUNT) as Median_amount_loan_debit_6M,
STDDEV(-AMOUNT) as STDDEV_amount_loan_debit_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_loan_debit_txns_6M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as loan_debit_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  loans = 1 AND TYPE='debit'
GROUP BY BUSINESS_ID),

aggregate_1_3_6_loan_debit as (
SELECT * 
FROM
sixMonths_loan_debit AS a
FULL OUTER JOIN 
threeMonths_loan_debit AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_loan_debit AS c
USING (BUSINESS_ID))

SELECT * FROM aggregate_1_3_6_loan_debit 
""")

In [76]:
features_df = features_df.merge(loan_debits, on='business_id', how='left')
features_df[loan_debits.columns[1:]] = features_df[loan_debits.columns[1:]].fillna(0)

In [77]:
features_df['ratio_loan_debit_total_debit_1m'] = (features_df['amount_loan_debit_1m'] / features_df['amount_debited_1m']).fillna(0)
features_df['ratio_loan_debit_total_debit_3m'] = (features_df['amount_loan_debit_3m'] / features_df['amount_debited_3m']).fillna(0)
features_df['ratio_loan_debit_total_debit_6m'] = (features_df['amount_loan_debit_6m'] / features_df['amount_debited_6m']).fillna(0)

In [78]:
query_obj = SQLQuery('snowflake')
insurance_txn = query_obj(r""" 
WITH txn as (
  		SELECT a.id AS transaction_id, a.BUSINESS_ID , a.amount, a.TYPE, a.MEDIUM, a.description, a.TRANSACTION_DATE,  
              CASE WHEN a.TYPE = 'credit' and a.DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    a.description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN a.TYPE ='debit' AND a.MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN a.type = 'credit'  AND a.medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN a.medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN a.TYPE = 'credit' AND a.description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, a.TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken, c.loans, c.insurance, c.tax
          FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a  
          JOIN PROD_DB.ADHOC.LENDING_NOVO_TXN_CATEGORIES_SAMPLE c ON c.transaction_id = a.id 
           INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 d on a.business_id = d.business_id
WHERE signed_on_X IS NOT NULL AND  a.status = 'active'  AND  daySinceLoanTaken <= 180 AND daySinceLoanTaken > 0
),

oneMonth_insurance_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_insurance_debit_1M,
AVG(-AMOUNT) as Avg_amount_insurance_debit_1M,
Median(-AMOUNT) as Median_amount_insurance_debit_1M,
STDDEV(-AMOUNT) as STDDEV_amount_insurance_debit_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_insurance_debit_txns_1M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as insurance_debit_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND insurance = 1 AND TYPE='debit'
GROUP BY BUSINESS_ID),

threeMonths_insurance_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_insurance_debit_3M,
AVG(-AMOUNT) as Avg_amount_insurance_debit_3M,
Median(-AMOUNT) as Median_amount_insurance_debit_3M,
STDDEV(-AMOUNT) as STDDEV_amount_insurance_debit_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_insurance_debit_txns_3M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as insurance_debit_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  insurance = 1 AND TYPE='debit'  
GROUP BY BUSINESS_ID),


sixMonths_insurance_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_insurance_debit_6M,
AVG(-AMOUNT) as Avg_amount_insurance_debit_6M,
Median(-AMOUNT) as Median_amount_insurance_debit_6M,
STDDEV(-AMOUNT) as STDDEV_amount_insurance_debit_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_insurance_debit_txns_6M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as insurance_debit_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  insurance = 1 AND TYPE='debit'
GROUP BY BUSINESS_ID),

oneMonth_insurance_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_insurance_credit_1M,
AVG(AMOUNT) as Avg_amount_insurance_credit_1M,
Median(AMOUNT) as Median_amount_insurance_credit_1M,
STDDEV(AMOUNT) as STDDEV_amount_insurance_credit_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_insurance_credit_txns_1M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as insurance_credit_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND insurance = 1 AND TYPE='credit'
GROUP BY BUSINESS_ID),

threeMonths_insurance_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_insurance_credit_3M,
AVG(AMOUNT) as Avg_amount_insurance_credit_3M,
Median(AMOUNT) as Median_amount_insurance_credit_3M,
STDDEV(AMOUNT) as STDDEV_amount_insurance_credit_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_insurance_credit_txns_3M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as insurance_credit_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  insurance = 1 AND TYPE='credit'  
GROUP BY BUSINESS_ID),

sixMonths_insurance_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_insurance_credit_6M,
AVG(AMOUNT) as Avg_amount_insurance_credit_6M,
Median(AMOUNT) as Median_amount_insurance_credit_6M,
STDDEV(AMOUNT) as STDDEV_amount_insurance_credit_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_insurance_credit_txns_6M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as insurance_credit_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  insurance = 1 AND TYPE='credit'
GROUP BY BUSINESS_ID),

aggregate_1_3_6_insurance_txns as (
SELECT * 
FROM
sixMonths_insurance_debit AS a
FULL OUTER JOIN 
threeMonths_insurance_debit AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_insurance_debit AS c
USING (BUSINESS_ID)
FULL OUTER JOIN
sixMonths_insurance_credit AS D
USING (BUSINESS_ID)
FULL OUTER JOIN 
threeMonths_insurance_credit AS e
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_insurance_credit AS f
USING (BUSINESS_ID))

SELECT * FROM aggregate_1_3_6_insurance_txns 
""")

In [79]:
features_df = features_df.merge(insurance_txn, on='business_id', how='left')
features_df[insurance_txn.columns[1:]] = features_df[insurance_txn.columns[1:]].fillna(0)

In [80]:
features_df['ratio_insurance_debit_total_debit_1m'] = (features_df['amount_insurance_debit_1m'] / features_df['amount_debited_1m']).fillna(0)
features_df['ratio_insurance_debit_total_debit_3m'] = (features_df['amount_insurance_debit_3m'] / features_df['amount_debited_3m']).fillna(0)
features_df['ratio_insurance_debit_total_debit_6m'] = (features_df['amount_insurance_debit_6m'] / features_df['amount_debited_6m']).fillna(0)
features_df['ratio_insurance_credit_total_credit_1m'] = (features_df['amount_insurance_credit_1m'] / features_df['amount_credited_1m']).fillna(0)
features_df['ratio_insurance_credit_total_credit_3m'] = (features_df['amount_insurance_credit_3m'] / features_df['amount_credited_3m']).fillna(0)
features_df['ratio_insurance_credit_total_credit_6m'] = (features_df['amount_insurance_credit_6m'] / features_df['amount_credited_6m']).fillna(0)

In [81]:
query_obj = SQLQuery('snowflake')
tax_txn = query_obj(r""" 
WITH txn as (
  		SELECT a.id AS transaction_id, a.BUSINESS_ID , a.amount, a.TYPE, a.MEDIUM, a.description, a.TRANSACTION_DATE,  
              CASE WHEN a.TYPE = 'credit' and a.DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                )
                AND NOT(
	                    a.description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                  CASE WHEN a.TYPE ='debit' AND a.MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                  CASE WHEN a.type = 'credit'  AND a.medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                  CASE WHEN a.medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                  CASE WHEN a.TYPE = 'credit' AND a.description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                    datediff(day, a.TRANSACTION_DATE, signed_on_x) AS daySinceLoanTaken, c.loans, c.insurance, c.tax
          FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" a  
          JOIN PROD_DB.ADHOC.LENDING_NOVO_TXN_CATEGORIES_SAMPLE c ON c.transaction_id = a.id 
           INNER JOIN  PROD_DB.ADHOC.PILOT_DATASET_3 d on a.business_id = d.business_id
WHERE signed_on_X IS NOT NULL AND  a.status = 'active'  AND  daySinceLoanTaken <= 180 AND daySinceLoanTaken > 0
),

oneMonth_tax_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_tax_debit_1M,
AVG(-AMOUNT) as Avg_amount_tax_debit_1M,
Median(-AMOUNT) as Median_amount_tax_debit_1M,
STDDEV(-AMOUNT) as STDDEV_amount_tax_debit_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_tax_debit_txns_1M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as tax_debit_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND tax = 1 AND TYPE='debit'
GROUP BY BUSINESS_ID),

threeMonths_tax_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_tax_debit_3M,
AVG(-AMOUNT) as Avg_amount_tax_debit_3M,
Median(-AMOUNT) as Median_amount_tax_debit_3M,
STDDEV(-AMOUNT) as STDDEV_amount_tax_debit_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_tax_debit_txns_3M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as tax_debit_size_3M
FROM txn 
WHERE daySinceLoanTaken <=90 AND  tax = 1 AND TYPE='debit'  
GROUP BY BUSINESS_ID),


sixMonths_tax_debit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(-AMOUNT) as amount_tax_debit_6M,
AVG(-AMOUNT) as Avg_amount_tax_debit_6M,
Median(-AMOUNT) as Median_amount_tax_debit_6M,
STDDEV(-AMOUNT) as STDDEV_amount_tax_debit_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_tax_debit_txns_6M,
SUM(-AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as tax_debit_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  tax = 1 AND TYPE='debit'
GROUP BY BUSINESS_ID),

oneMonth_tax_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_tax_credit_1M,
AVG(AMOUNT) as Avg_amount_tax_credit_1M,
Median(AMOUNT) as Median_amount_tax_credit_1M,
STDDEV(AMOUNT) as STDDEV_amount_tax_credit_1M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_tax_credit_txns_1M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as tax_credit_size_1M
FROM txn 
WHERE daySinceLoanTaken <=30 AND tax = 1 AND TYPE='credit'
GROUP BY BUSINESS_ID),

threeMonths_tax_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_tax_credit_3M,
AVG(AMOUNT) as Avg_amount_tax_credit_3M,
Median(AMOUNT) as Median_amount_tax_credit_3M,
STDDEV(AMOUNT) as STDDEV_amount_tax_credit_3M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_tax_credit_txns_3M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as tax_credit_size_3M
FROM txn 
WHERE daySinceLoanTaken <= 90 AND  tax = 1 AND TYPE='credit'  
GROUP BY BUSINESS_ID),

sixMonths_tax_credit as (SELECT 
BUSINESS_ID as BUSINESS_ID,
SUM(AMOUNT) as amount_tax_credit_6M,
AVG(AMOUNT) as Avg_amount_tax_credit_6M,
Median(AMOUNT) as Median_amount_tax_credit_6M,
STDDEV(AMOUNT) as STDDEV_amount_tax_credit_6M,
COUNT(DISTINCT TRANSACTION_ID) as distinct_tax_credit_txns_6M,
SUM(AMOUNT)/COUNT(DISTINCT TRANSACTION_ID) as tax_credit_size_6M
FROM txn 
WHERE daySinceLoanTaken <=180 AND  tax = 1 AND TYPE='credit'
GROUP BY BUSINESS_ID),

aggregate_1_3_6_tax_txns as (
SELECT * 
FROM
sixMonths_tax_debit AS a
FULL OUTER JOIN 
threeMonths_tax_debit AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_tax_debit AS c
USING (BUSINESS_ID)
FULL OUTER JOIN
sixMonths_tax_credit AS D
USING (BUSINESS_ID)
FULL OUTER JOIN 
threeMonths_tax_credit AS e
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_tax_credit AS f
USING (BUSINESS_ID))

SELECT * FROM aggregate_1_3_6_tax_txns 
""")

In [82]:
features_df = features_df.merge(tax_txn, on='business_id', how='left')
features_df[tax_txn.columns[1:]] = features_df[tax_txn.columns[1:]].fillna(0)

In [83]:
features_df['ratio_tax_debit_total_debit_1m'] = (features_df['amount_tax_debit_1m'] / features_df['amount_debited_1m']).fillna(0)
features_df['ratio_tax_debit_total_debit_3m'] = (features_df['amount_tax_debit_3m'] / features_df['amount_debited_3m']).fillna(0)
features_df['ratio_tax_debit_total_debit_6m'] = (features_df['amount_tax_debit_6m'] / features_df['amount_debited_6m']).fillna(0)
features_df['ratio_tax_credit_total_credit_1m'] = (features_df['amount_tax_credit_1m'] / features_df['amount_credited_1m']).fillna(0)
features_df['ratio_tax_credit_total_credit_3m'] = (features_df['amount_tax_credit_3m'] / features_df['amount_credited_3m']).fillna(0)
features_df['ratio_tax_credit_total_credit_6m'] = (features_df['amount_tax_credit_6m'] / features_df['amount_credited_6m']).fillna(0)

In [84]:
features_df['created_at'] = pd.Timestamp.now()

In [85]:
features_df.to_csv('./lending_novo_txn_features_postscreen_v1.csv', index=False)

In [86]:
features_df = pd.read_csv('./lending_novo_txn_features_postscreen_v1.csv')

In [87]:
import numpy as np
features_df = features_df.replace([np.inf, -np.inf], 0)

In [88]:
features_df.columns

Index(['business_id', 'email_sent_at', 'signed_on_x', 'fpd_plus_3',
       'past_1_month', 'cutoff_date', 'date_of_refresh', 'is_pd',
       'ratio_pd_freq_90_180', 'ratio_pd_amt_90_180',
       ...
       'stddev_amount_tax_credit_1m', 'distinct_tax_credit_txns_1m',
       'tax_credit_size_1m', 'ratio_tax_debit_total_debit_1m',
       'ratio_tax_debit_total_debit_3m', 'ratio_tax_debit_total_debit_6m',
       'ratio_tax_credit_total_credit_1m', 'ratio_tax_credit_total_credit_3m',
       'ratio_tax_credit_total_credit_6m', 'created_at'],
      dtype='object', length=353)

In [89]:
features_df.dtypes

business_id                          object
email_sent_at                        object
signed_on_x                          object
fpd_plus_3                            int64
past_1_month                        float64
                                     ...   
ratio_tax_debit_total_debit_6m      float64
ratio_tax_credit_total_credit_1m    float64
ratio_tax_credit_total_credit_3m    float64
ratio_tax_credit_total_credit_6m    float64
created_at                           object
Length: 353, dtype: object

In [90]:
engine = SQLQuery('snowflake').engine
features_df.to_sql('lending_novo_txn_features_postscreen_v1', engine, chunksize=100,
                   if_exists='append', index=False, method='multi')

In [91]:
engine = SQLQuery('snowflake').engine
features_df.to_sql('lending_novo_txn_features_model_postscreen_v2', engine, chunksize=100,
                   if_exists='replace', index=False, method='multi')